<a href="https://colab.research.google.com/github/krdlnsnks/Miuul/blob/main/CRM_MFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving online_retail_II.xlsx to online_retail_II.xlsx


In [ ]:
import io
df = pd.read_excel(io.BytesIO(uploaded['online_retail_II.xlsx']))

In [ ]:
print(df.head())
print(df.shape)

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate   Price  Customer ID         Country  
0 2009-12-01 07:45:00 6.95000  13085.00000  United Kingdom  
1 2009-12-01 07:45:00 6.75000  13085.00000  United Kingdom  
2 2009-12-01 07:45:00 6.75000  13085.00000  United Kingdom  
3 2009-12-01 07:45:00 2.10000  13085.00000  United Kingdom  
4 2009-12-01 07:45:00 1.25000  13085.00000  United Kingdom  
(525461, 8)


In [ ]:
df["Description"].nunique()

4681

In [ ]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [ ]:
df.groupby("Description").agg({"Quantity": "sum"}).head()

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


In [ ]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [ ]:
df["Invoice"].nunique()

28816

In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()

,TotalPrice
Invoice,
489434,505.30000
489435,145.80000
489436,630.33000
489437,310.75000
489438,2286.24000


In [ ]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(417534, 9)

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.00000,12.75881,101.22042,-9360.00000,2.00000,4.00000,12.00000,19152.00000
Price,417534.00000,3.88755,71.13180,0.00000,1.25000,1.95000,3.75000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,13983.00000,15311.00000,16799.00000,18287.00000
TotalPrice,417534.00000,19.99408,99.91586,-25111.09000,4.25000,11.25000,19.35000,15818.40000


In [ ]:
df=df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [ ]:
df["Invoice"].max()

538171

In [ ]:
today_date=dt.datetime(2010, 12, 11)
type(today_date)

datetime.datetime

In [ ]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [ ]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000


In [ ]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [ ]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.00000,91.26982,96.94430,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000


In [ ]:
rfm = rfm[rfm["monetary"] > 0]

In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.00000,91.17254,96.86146,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4312.00000,4.45571,8.17021,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4312.00000,2048.23824,8914.48128,2.95000,307.98750,706.02000,1723.14250,349164.35000


In [ ]:
rfm.shape

(4312, 3)

In [ ]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])

In [ ]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

In [ ]:
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1,2,3,4,5])

In [ ]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str)+
                    rfm['frequency_score'].astype(str))

In [ ]:
rfm[rfm["RFM_SCORE"] == "55"]

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12415.00000,11,7,19543.84000,5,5,5,55
12431.00000,9,13,4370.52000,5,5,5,55
12471.00000,10,49,20139.74000,5,5,5,55
12472.00000,5,13,11308.48000,5,5,5,55
12474.00000,14,13,5048.66000,5,5,5,55
12483.00000,14,11,2635.60000,5,5,5,55
12490.00000,14,11,4405.71000,5,5,5,55
12523.00000,10,12,2582.38000,5,5,5,55
12540.00000,14,16,8854.25000,5,5,5,55


In [ ]:
rfm[rfm["RFM_SCORE"] == "11"]

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12355.00000,203,1,488.21000,1,2,1,11
12362.00000,374,1,130.00000,1,1,1,11
12366.00000,269,1,500.24000,1,2,1,11
12368.00000,264,1,917.70000,1,3,1,11
12378.00000,198,1,1407.70000,1,4,1,11
12392.00000,227,1,234.75000,1,1,1,11
12396.00000,310,1,931.43000,1,3,1,11
12404.00000,318,1,63.24000,1,1,1,11
12416.00000,292,1,202.56000,1,1,1,11


In [ ]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'cant loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions',
}

In [ ]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [ ]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency         monetary      
                         mean count      mean count       mean count
segment                                                             
about_to_sleep       53.81924   343   1.20117   343  441.32000   343
at risk             152.15876   611   3.07365   611 1188.87832   611
cant loose          124.11688    77   9.11688    77 4099.45000    77
champions             7.11916   663  12.55354   663 6852.26417   663
hibernating         213.88571  1015   1.12611  1015  403.97784  1015
loyal_customers      36.28706   742   6.83019   742 2746.06735   742
need_attention       53.26570   207   2.44928   207 1060.35700   207
new_customers         8.58000    50   1.00000    50  386.19920    50
potential_loyalists  18.79304   517   2.01741   517  729.51099   517
promising            25.74713    87   1.00000    87  367.08678    87

In [ ]:
new_df = pd.DataFrame()

In [ ]:
new_df["new_customer_id"]=rfm[rfm["segment"]=="new_customers"].index

In [ ]:
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [ ]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,165,11,372.86000,2,2,5,25,cant loose
12347.00000,3,2,1323.32000,5,4,2,52,potential_loyalists
12348.00000,74,1,222.16000,2,1,1,21,hibernating
12349.00000,43,3,2671.14000,3,5,3,33,need_attention
12351.00000,11,1,300.93000,5,2,1,51,new_customers


In [ ]:
new_df.head()

,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441
